In [930]:
#%matplotlib ipympl
import numpy as np
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import os
import openpyxl 
from scipy.integrate import simps
import adi
import pandas as pd
from tqdm import tqdm
from scipy.stats import mannwhitneyu
from scipy.ndimage import gaussian_filter1d

In [931]:
font_path = 'C:\\Windows\\Fonts\\simsun.ttc'  # 宋體
font_prop = FontProperties(fname=font_path)

mode = 'save'   # 'show' or 'save'

In [932]:
def butter(DataL, DataR, cut_low, cut_high, sample_rate, order):
    nyqs = sample_rate * 0.5
    H_cut = cut_high / nyqs
    L_cut = cut_low / nyqs
    sos = signal.butter(order, [L_cut, H_cut], analog=False, btype='bandpass', output='sos')
    Filter_Left = signal.sosfiltfilt(sos, DataL)
    Filter_Right = signal.sosfiltfilt(sos, DataR)
    return Filter_Left, Filter_Right

In [933]:
def bassel(DataL, DataR, cut_low, cut_high, sample_rate, order):
    #貝塞爾
    nyqs = sample_rate * 0.5
    H_cut = cut_high / nyqs
    L_cut = cut_low / nyqs
    sos=signal.bessel(order, [L_cut, H_cut] ,  btype='bandpass',  analog=False,  output='sos')
    Filter_Left = signal.sosfiltfilt(sos,  DataL) 
    Filter_Right = signal.sosfiltfilt(sos,  DataR) 

    return Filter_Left, Filter_Right

In [934]:
def find_peak(Filter_Data):
    valley_x, valley_y = find_peaks(Filter_Data * -1, height=0, distance=500)
    cardiac_cycle = np.diff(valley_x)
    peaks_x, peaks_y = find_peaks(Filter_Data, height=0, distance=500)
    peaks_y = peaks_y['peak_heights']
    valley_y = valley_y['peak_heights']
    return peaks_x, peaks_y, valley_x, valley_y, cardiac_cycle

In [935]:
def derivative(Data, Level, values=[]):
    result = np.gradient(Data)

    if Level == 0:
        values.append(Data)
        return 0
    else:
        values.append(Data)
        return derivative(result, Level - 1, values)
    
def process_wave(cycle):
    values = []
    derivative(cycle, 3, values)
    values = np.array(values)
    origin, derivative_1, derivative_2, derivative_3 = values
    derivative_1 = derivative_1 * 50
    derivative_2 = derivative_2 * 5000
    derivative_3 = derivative_3 * 100000

    return [derivative_1, derivative_2, derivative_3]

In [936]:
def Find_Path(path):

    File_path = []

    #find all Data_file path 
    for root,  subfolders,  filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('.txt'):
                continue
            filepath = root +'/'+ filename
            new_filepath = filepath.replace("\\", "/")
            File_path.append(new_filepath)

    return File_path

In [937]:
def get_Imformation(path,locate, imformation=[]):
    test=path.split('/')
    Name = test[locate[0]]
    Date = test[locate[1]]
    State_check = test[locate[2]]
    if State_check =='易堵':
        State = '0'
    else: 
        State = '1'

    file_name = test[len(test)-1]
    name_check = file_name.find('R')
    if name_check != -1:
        Status = 'Right'
    else:
        Status = 'Left'
    
    imformation =[Name, Date, State, Status]

    return imformation, Name

In [938]:
def plot(cycle_1, cycle_2, parameter, Name,i):
    p = 0
    def on_key(event):
        if event.key == 'z':
            plt.close()
    if p == 0:
        plt.plot(cycle_1, label='Left PPG')
        plt.plot(cycle_2, label='Right PPG')
    if p == 1:
        print('not yet')

    plt.title(f'{Name}, {i + 1}th Left_Right',fontproperties=font_prop)
    plt.legend()
    plt.grid()
    if mode == 'show':
        plt.show()
    else:
        plt.savefig(f'F:\\Python\\PPG\\Cycle\\{Name}, {i + 1}th Left_Right.jpg')

    plt.gcf().canvas.mpl_connect('key_press_event', on_key)
    plt.close()

In [939]:
def plot_d1(derivative, Name, i, Feature):
    
    x = np.linspace(0, len(derivative[0]), len(derivative[0]))
    plt.figure()
    plt.plot(derivative[0])
    plt.plot(x[Feature], derivative[0][Feature], '*', label='Peak')
    plt.title(f'{Name}, {i + 1}th Left_Right',fontproperties=font_prop)
    plt.legend()
    plt.grid()
    if mode == 'show':
        plt.show()
    else:
        plt.savefig(f'F:\\Python\\PPG\\Cycle\\{Name}, {i + 1}th Left_Right.jpg')
    #plt.show()

In [940]:
def calculate_d1(derivative, Name, i, Feature):
    #derivative = process_wave(cycle)
    
    d1_peak_x, d1_peak_y = find_peaks(derivative[0], height=0, distance=800)
    d1_valley_x,d1_valley_y = find_peaks(derivative[0] * -1, height=0, distance=800)

    d1_peak_y = d1_peak_y['peak_heights']
    d1_valley_y = d1_valley_y['peak_heights']

    print(d1_peak_y, d1_valley_y)


    feature = [d1_peak_x, d1_valley_x]
    # plot_d1(derivative, Name, i, feature)


In [941]:
def plot_d2(origin, derivative, Name, i, Feature, hands):
    
    x = np.linspace(0, len(derivative[1]), len(derivative[1]))
    plt.figure()
    
    plt.plot(origin)
    plt.plot(derivative[1])
    plt.plot(x[Feature], derivative[1][Feature], '*', label='Peak')
    plt.title(f'{Name}, {i + 1}th {hands}',fontproperties=font_prop)
    plt.legend()
    plt.grid()
    if mode == 'show':
        plt.show()
    else:
        plt.savefig(f'F:\\TDPPG\\{Name}, {i + 1}th {hands}.jpg')
        plt.close()

d2

In [942]:
def calculate_d2(origin, derivative, Name, i, hands):
    TDPPG_x = np.where(np.diff(np.sign(derivative[2])))[0]
    # 設置最小距離（例如，至少 10 個數據點）
    min_distance = 30

    def filter_by_distance(zero_crossings, min_distance):
        filtered_crossings = []
        last_index = -min_distance  # 初始化為一個遠距離的負數
        
        for index in zero_crossings:
            if index - last_index >= min_distance:
                filtered_crossings.append(index)
                last_index = index
        
        return np.array(filtered_crossings)

    # 過濾零點
    TDPPG_x = filter_by_distance(TDPPG_x, min_distance)

    closest_indices = []

    a_point = np.array(find_peaks(derivative[1], height=0.6, distance=900)[0])


    for num in a_point:
        differences = np.abs(TDPPG_x - num)
        closest_index = np.argmin(differences)
        closest_indices.append(closest_index)
    if len(closest_indices) < 2:  #判斷抓到兩個周期
        plot_d2(origin, derivative, Name, i, TDPPG_x, hands)
        print(f'{Name}, {i + 1}th',TDPPG_x[closest_indices])
        return 0 
    else:
        TDPPG_x_new = TDPPG_x[closest_indices[0]:closest_indices[0]+6]
        TDPPG_x_new = np.append(TDPPG_x_new, TDPPG_x[closest_indices[1]:closest_indices[1]+6])
        if TDPPG_x_new[6] - TDPPG_x_new[5] < TDPPG_x_new[6] * 0.3: #判斷是否抓錯feature
            plot_d2(origin, derivative, Name, i, TDPPG_x_new, hands)
            print(f'{Name}, {i + 1}th {hands} cant find the feature')
            return 0


    if mode =='show':
        plot_d2(origin,derivative, Name, i, TDPPG_x_new, hands)
        print(TDPPG_x_new)
    else:
        np.savetxt(f'F:\\Patient_signal\\{Name}, {i + 1}th {hands} d2 feature.txt', TDPPG_x_new, fmt='%d', delimiter=' ', newline=' ')
        
    #plot_d2(origin, derivative, Name, i, TDPPG_x_new, hands)
    return 1


In [943]:
def resize_wave(waveform, Name, i, hands, target_length=2000):
    quilty = 1

    derivative = process_wave(waveform)

    resized = signal.resample(waveform, target_length)
    
    for j in range(3):
        derivative[j] = signal.resample(derivative[j], target_length)


    # record = dict() #!需要一個dict紀錄每個特徵點
    # record[f'{Name}, {i + 1}th Left'] = 100
    # record[f'{Name}, {i + 1}th Right'] = 100
    Feature = []

    #quilty = calculate_d1(derivative, Name, i, Feature)
    
    quilty = calculate_d2(resized, derivative, Name, i, hands)

    if mode == 'save' and quilty == 1:
        np.save(f'F:\\Patient_signal\\{Name}, {i + 1}th {hands}.npy', resized)
        #np.save(f'F:\\Patient_signal\\{Name}, {i + 1}th {hands} d1.npy', L_derivative[0])
        np.save(f'F:\\Patient_signal\\{Name}, {i + 1}th {hands} d2.npy', derivative[1])

In [944]:
def main(): #!for folder data

    patient = 'normal' #!normal or patient
    
    channel1_id = 2
    channel2_id = 4
    record_id = 1

    if patient == 'normal':
        File_path = Find_Path("F:\\正常人Data") #!正常人
    else:
        File_path = Find_Path("F:\\第二次收案") #!病患
    print('找到資料筆數', len(File_path))

    Features = []
    df_c = pd.DataFrame()
    for j, path in tqdm(enumerate(File_path), total=len(File_path), desc='Processing'):
        Data = adi.read_file(path)

        Right = Data.channels[channel1_id - 1].get_data(record_id)
        Left = Data.channels[channel2_id - 1].get_data(record_id)

        Filter_Left,Filter_Right = butter(Left, Right, 0.5, 9, 1000, 4)

        L_wave = Filter_Left[100000:120000] * 10
        R_wave = Filter_Right[100000:120000] * 10

        L_valley_x, L_valley_y = find_peaks(L_wave * -1, height=0, distance=150)
        R_valley_x, R_valley_y = find_peaks(R_wave * -1, height=0, distance=150)

        L_valley_y = L_valley_y['peak_heights']
        R_valley_y = R_valley_y['peak_heights']

        if patient == 'normal':
            Imformation,Name = get_Imformation(path,locate=[2,3,4]) #!正常人
        else:
            Imformation,Name = get_Imformation(path,locate=[3,4,2]) #!病患

        if len(L_valley_x) > len(R_valley_x): #找最小的cycle
            min_cycle = len(R_valley_x)
        else:
            min_cycle = len(L_valley_x)
        
        for i in range(0,min_cycle - 2,2):
            diff = np.abs(L_valley_x[i] - R_valley_x[i]) #time diff
            L_cycle = L_wave[L_valley_x[i]:L_valley_x[i + 2]] #two cycle
            L_cycle_cut = [L_wave[L_valley_x[i]:L_valley_x[i + 1]], L_wave[L_valley_x[i + 1]:L_valley_x[i + 2]]] #divide 2

            L_peaks_x, L_peaks_y = find_peaks(L_cycle, height=0, distance=500)
            L_peaks_y = L_peaks_y['peak_heights']
            L_peak = [L_peaks_x, L_peaks_y]

            R_cycle = R_wave[L_valley_x[i]:L_valley_x[i + 2]]
            R_cycle_cut = [R_wave[R_valley_x[i]:R_valley_x[i + 1]], R_wave[R_valley_x[i + 1]:R_valley_x[i + 2]]]  # vivide 2

            R_peaks_x, R_peaks_y = find_peaks(R_cycle, height=0, distance=500)
            R_peaks_y = R_peaks_y['peak_heights']
            R_peak = [R_peaks_x, R_peaks_y]

            if len(L_cycle) < 1100 or len(L_peaks_y) != 2 or len(R_peaks_y) != 2 or len(R_cycle) < 1100:
                continue

            if L_peaks_y[0] < 0.5:
                L_cycle *= 0.5 / L_peaks_y[0]
                L_peaks_y[0] = 0.5
                L_peaks_y[1] = 0.5

            if R_peaks_y[0] < 0.5:
                R_cycle *= 0.5 / R_peaks_y[0]
                R_peaks_y[0] = 0.5
                R_peaks_y[1] = 0.5
            resize_wave(L_cycle, Name, i,"Left", 2000 )
            resize_wave(R_cycle, Name, i, "Right", 2000 )



In [945]:
# def main(): #!for single data
#     channel1_id = 2
#     channel2_id = 4
#     record_id = 1

#     Name = 'test'
    
#     path = "F:\\正常人Data\\Normal1\\四下午\\Normal1.adicht"
#     Data = adi.read_file(path)

#     Right = Data.channels[channel1_id - 1].get_data(record_id)
#     Left = Data.channels[channel2_id - 1].get_data(record_id)

#     Filter_Left,Filter_Right = butter(Left, Right, 0.5, 9, 1000, 4)

#     L_wave = Filter_Left[100000:120000] * 10
#     R_wave = Filter_Right[100000:120000] * 10

#     L_valley_x, L_valley_y = find_peaks(L_wave * -1, height=0, distance=150)
#     R_valley_x, R_valley_y = find_peaks(R_wave * -1, height=0, distance=150)

#     L_valley_y = L_valley_y['peak_heights']
#     R_valley_y = R_valley_y['peak_heights']

#     if len(L_valley_x) > len(R_valley_x): #找最小的cycle
#         min_cycle = len(R_valley_x)
#     else:
#         min_cycle = len(L_valley_x)
    
#     for i in range(0,min_cycle - 2,2):
#         diff = np.abs(L_valley_x[i] - R_valley_x[i]) #time diff
#         L_cycle = L_wave[L_valley_x[i]:L_valley_x[i + 2]] #two cycle
#         L_cycle_cut = [L_wave[L_valley_x[i]:L_valley_x[i + 1]], L_wave[L_valley_x[i + 1]:L_valley_x[i + 2]]] #divide 2

#         L_peaks_x, L_peaks_y = find_peaks(L_cycle, height=0, distance=500)
#         L_peaks_y = L_peaks_y['peak_heights']
#         L_peak = [L_peaks_x, L_peaks_y]

#         R_cycle = R_wave[L_valley_x[i]:L_valley_x[i + 2]]
#         R_cycle_cut = [R_wave[R_valley_x[i]:R_valley_x[i + 1]], R_wave[R_valley_x[i + 1]:R_valley_x[i + 2]]]  # vivide 2

#         R_peaks_x, R_peaks_y = find_peaks(R_cycle, height=0, distance=500)
#         R_peaks_y = R_peaks_y['peak_heights']
#         R_peak = [R_peaks_x, R_peaks_y]

#         if len(L_cycle) < 1100 or len(L_peaks_y) != 2 or len(R_peaks_y) != 2 or len(R_cycle) < 1100:
#             continue

#         if L_peaks_y[0] < 0.5:
#             L_cycle *= 0.5 / L_peaks_y[0]
#             L_peaks_y[0] = 0.5
#             L_peaks_y[1] = 0.5

#         if R_peaks_y[0] < 0.5:
#             R_cycle *= 0.5 / R_peaks_y[0]
#             R_peaks_y[0] = 0.5
#             R_peaks_y[1] = 0.5
        

#         resize_wave(L_cycle, R_cycle, Name, i, 2000)


In [946]:
if __name__ == '__main__':
    main()

找到資料筆數 63


Processing:   0%|          | 0/63 [00:00<?, ?it/s]

Normal1, 5th Right cant find the feature
Normal1, 9th Right cant find the feature


Processing:  11%|█         | 7/63 [00:00<00:05,  9.35it/s]

Normal15, 5th Right cant find the feature
Normal15, 13th Right cant find the feature
Normal17, 13th Left cant find the feature
Normal17, 13th Right cant find the feature
Normal17, 19th Left cant find the feature


...

Normal65, 3th Left cant find the feature
Normal65, 3th Right cant find the feature
Normal65, 15th Right cant find the feature
Normal7, 7th Left cant find the feature
Normal7, 7th Right cant find the feature
Normal7, 23th Left cant find the feature
Normal7, 23th Right cant find the feature
Normal7, 27th Left cant find the feature
Normal7, 27th Right cant find the feature
Normal7, 29th Left cant find the feature
Normal7, 29th Right cant find the feature
Normal7, 35th Left cant find the feature


Processing:  97%|█████████▋| 61/63 [00:18<00:00,  3.02it/s]

Normal7, 35th Right cant find the feature
Normal9, 1th Left cant find the feature
Normal9, 1th Right cant find the feature
Normal9, 3th Left cant find the feature
Normal9, 3th Right cant find the feature
Normal9, 5th Left cant find the feature
Normal9, 5th Right cant find the feature
Normal9, 7th Left cant find the feature
Normal9, 9th Left cant find the feature
Normal9, 9th Right cant find the feature
Normal9, 11th Left cant find the feature
Normal9, 11th Right cant find the feature
Normal9, 13th Left cant find the feature
Normal9, 13th Right cant find the feature
Normal9, 15th Left cant find the feature
Normal9, 17th Left cant find the feature
Normal9, 17th Right cant find the feature
Normal9, 19th Left cant find the feature
Normal9, 19th Right cant find the feature
Normal9, 21th Left cant find the feature
Normal9, 21th Right cant find the feature
Normal9, 23th Left cant find the feature
Normal9, 23th Right cant find the feature
Normal9, 25th Left cant find the feature
Normal9, 25th 

Processing: 100%|██████████| 63/63 [00:21<00:00,  2.99it/s]

Normal9, 29th Right cant find the feature


In [947]:
'''
65 3th
16 7th
16 1th
15 1th
32 7th
31 3th
29 5th
55 all
51 all
5 all
48 all
42 can't use 
41 can't use
3 all
27 all

'''

"\n65 3th\n16 7th\n16 1th\n15 1th\n32 7th\n31 3th\n29 5th\n55 all\n51 all\n5 all\n48 all\n42 can't use \n41 can't use\n3 all\n27 all\n\n"